In [2]:
import voila
import ipywidgets as ipw
from ipywidgets import interact
import plotly.express as px
from string import ascii_uppercase as upstr
import json

In [3]:
img_path = open('AbSciImg.jfif', 'rb')
img = img_path.read()
ipw.Image(value=img, format='jfif', height='200px', width='200px')

Image(value=b'\xff\xd8\xff\xe0\x00\x10JFIF\x00\x01\x01\x00\x00\x01\x00\x01\x00\x00\xff\xfe\x00;CREATOR: gd-jpe…

# Parser Input Form

In [4]:
# !pip install voila

In [5]:
# !jupyter nbextension enable --py widgetsnbextension --sys-prefix
# !jupyter serverextension enable voila --sys-prefix

In [6]:
# !pip install jslinks

In [7]:
STYLE = {'description_width': 'initial'}

In [8]:
proj_dict = {}

In [9]:
with open('parser_data.json','w') as parser_file:
    json.dump(proj_dict, parser_file, indent=4)

### Enter Project Name

In [10]:
proj_name = ipw.Text()
proj_name

Text(value='')

### Enter plate ids, e.g. 'P1-1, P1-2, P2...' using '-' for any replicate, or 'P1, P2' if not replicates

In [11]:
plates = ipw.Text()
plates

Text(value='')

### Choose if run is four or eight point run.

In [12]:
point_choice = ipw.Dropdown(options=[('Four', 4), ('Eight', 8)])
point_choice

Dropdown(options=(('Four', 4), ('Eight', 8)), value=4)

### Enter the dilution volumes for this project

In [13]:
d_vols = ipw.Text()
d_vols

Text(value='')

### Click checkbox to include OD File with project

In [14]:
x = ipw.Checkbox(indent=False)
x

Checkbox(value=False, indent=False)

### If you added Standard Curves, fill in information below
##### Note that if you are adding multiple projects, you need to 'Add Standard' for each project! Standard data will not carry over!

In [15]:
std_dict_all = {}

#### Select standard location, e.g., 'Column' or 'Row'

In [16]:
loc = ipw.Dropdown(options=["Column", "Row"])
loc

Dropdown(options=('Column', 'Row'), value='Column')

#### Choose the starting well id, e.g. A11, or G01

In [17]:
well_loc = ipw.Text()
well_loc

Text(value='')

#### Enter the standard concentrations. Make sure your first concentration aligns with starting well id indicated!

In [18]:
std_conc = ipw.Text()
std_conc

Text(value='')

In [19]:
stc_out = ipw.Output()
stc_out

Output()

In [20]:
add_stc_button = ipw.Button(description='Add Standard', button_style='info')

@stc_out.capture()
def add_standard(event):
    global std_dict_all
    stc_out.clear_output()
    std_conc_list = [float(x) for x in std_conc.value.split(',')]
    std_conc_list_len = len(std_conc_list)
    if loc.value == 'Column':
        col_letter = well_loc.value[:1]
        letter_idx = upstr.index(col_letter)
        col_num = well_loc.value[1:]
        std_ids = [f"{upstr[letter]}{str(col_num).zfill(2)}" for letter in range(letter_idx, std_conc_list_len)]
        std_dict = (dict(zip(std_ids, std_conc_list)))

    elif loc.value == 'Row':
        row_letter = well_loc.value[:1]
        row_num = int(well_loc.value[1:])
        std_ids = [f"{row_letter}{str(num).zfill(2)}" for num in range(row_num, row_num + std_conc_list_len)]
        std_dict = dict(zip(std_ids, std_conc_list))
    try:
        std_dict_all.update(std_dict)
    except AttributeError:
        std_dict_all = {}
        std_dict_all.update(std_dict)
    else:
        for key, value in std_dict_all.items():
            print(f'Well ID: {key}, Concentration: {value}')
        print("\nTo add replicate, change well id, or 'column/row' to indicate new standard curve position.\n")
        return std_dict_all

add_stc_button.on_click(add_standard)
add_stc_button

Button(button_style='info', description='Add Standard', style=ButtonStyle())

### Add project when finished and check for accuracy!

In [24]:
button_update = ipw.Button(
    description='Add Project',
    button_style='info'
)
output = ipw.Output()
def on_button_click(event):
    global std_dict_all
    output.clear_output()
    
    with open('parser_data.json', 'r') as parser_file:
        proj_dict_all = json.load(parser_file)
    
    proj = proj_name.value
    point = point_choice.value
    plate_ids = list(map(str.strip, plates.value.split(',')))
    d_volumes = [float(x) for x in d_vols.value.split(',')]

    proj_dict = {proj:{
        'plates': plate_ids,
        'points': point,
        'volumes': d_volumes,
        'od_file': x.value,
        'std_conc': std_dict_all
    }
                }
    proj_dict_all.update(proj_dict)
    
    with open('parser_data.json', 'w') as parser_file:
        json.dump(proj_dict_all, parser_file, indent=4)
        
    std_dict_all = {}
    
    with output:
        print('\nProject Updated')
        for proj, inner in proj_dict_all.items():
            print(f'\nProject name entered: {proj}\n')
            print(f"Plate Ids: {', '.join(inner['plates'])}")
            print(f"Point Scheme: {inner['points']}")
            print(f"Dilution Volumes: {d_vols.value}")
            print(f"Add OD data: {inner['od_file']}")
            for key, value in inner['std_conc'].items():
                print(f"Well ID: {key}, Standard Concentration: {value}")
#             for key, value in inner.items():
#                 print(f'{key} entered: {value}\n')
    return std_dict_all
button_update.on_click(on_button_click)
vbox_result = ipw.VBox([button_update, output])
vbox_result